In [ ]:
from datetime import datetime
import plotly.graph_objects as pg
import plotly.express as px
import plotly.io as pio
import pandas as pd
import locale
locale.setlocale(locale.LC_TIME, "")


from cashd import db

CORES = ["#478eff", "gray"]

Este notebook será usado para experimentar e desenvolver visualizações de maneira mais eficiente.

In [ ]:
periodo = "sem"
n = 10

tbl = db.saldos_transac_periodo(periodo=periodo, n=n)
tbl["Data"].apply(lambda x: datetime.strptime(x+"-0", "%Y-%W-%w"))

In [ ]:
### balancos_por_periodo
periodo = "dia"
n = 30

tbl = db.saldos_transac_periodo(periodo=periodo, n=n)
if periodo == "sem":
    tbl["Data"] = tbl["Data"].apply(lambda x: datetime.strptime(x+"-0", "%Y-%W-%w"))
tbl["Data"] = pd.to_datetime(tbl.Data)
tbl["SomasDisplay"] = tbl["Somas"]\
    .apply(lambda x: f"{x:_.2f}".replace(".", ",").replace("_", " "))
tbl["AbatDisplay"] = tbl["Abatimentos"]\
    .apply(lambda x: f"{x:_.2f}".replace(".", ",").replace("_", " "))

datestr = "%B de %Y"
if periodo == "dia":
    datestr = "%d de %B de %Y"
elif periodo == "sem":
    datestr = "%Y, semana %W"

layout = pg.Layout(
    margin=dict(l=0, r=0, t=0, b=0),
    template="none",
    showlegend=False,
    hovermode="x unified",
    xaxis=dict(
        tickmode="array",
        tickvals=[i for i in tbl["Data"]],
        ticktext=[i.strftime(datestr) for i in tbl["Data"]]
    )
)

fig = pg.Figure(layout=layout)
fig.add_trace(pg.Bar(
    x=tbl["Data"], y=tbl["Somas"], name="Somas",
    customdata=tbl[["SomasDisplay"]],
    hovertemplate="<b>R$ %{customdata[0]}</b>",
    offsetgroup=0,
    marker=dict(color=CORES[1])
))
fig.add_trace(pg.Bar(
    x=tbl["Data"], y=tbl["Abatimentos"], name="Abatimentos",
    customdata=tbl[["AbatDisplay"]],
    hovertemplate="<b>R$ %{customdata[0]}</b>",
    offsetgroup=0,
    marker=dict(color=CORES[0])
))
fig.show(config={"displayModeBar": False})

In [ ]:
### saldos_por_periodo
periodo = "sem"
n = 30

tbl = db.saldos_transac_periodo(periodo=periodo)
if periodo == "sem":
    tbl["Data"] = tbl["Data"].apply(lambda x: datetime.strptime(x+"-0", "%Y-%W-%w"))
tbl["Data"] = pd.to_datetime(tbl.Data)

tbl["SaldoAcum"] = (tbl["Somas"] + tbl["Abatimentos"]).cumsum()
tbl["SaldoAcumDisplay"] = tbl["SaldoAcum"]\
    .apply(lambda x: f"{x:_.2f}".replace(".", ",").replace("_", " "))

if n:
    tbl = tbl.tail(n)

datestr = "%B de %Y"
if periodo == "dia":
    datestr = "%d de %B de %Y"
elif periodo == "sem":
    datestr = "%Y, semana %W"

layout = pg.Layout(
    showlegend=False,
    template="plotly_white",
    hovermode="x unified",
    xaxis=dict(
        tickmode="array",
        tickvals=[i for i in tbl["Data"]],
        ticktext=[i.strftime(datestr) for i in tbl["Data"]],
        showticklabels=False
    ),
    yaxis_tickprefix = 'R$',
    yaxis_tickformat=" "
)

fig = pg.Figure(layout=layout)
fig.add_trace(pg.Scatter(
    x=tbl["Data"], y=tbl["SaldoAcum"], name="Saldo", 
    mode="lines+markers", 
    customdata=tbl[["SaldoAcumDisplay"]],
    hovertemplate="<b>R$ %{customdata[0]}</b>",
    offsetgroup=0,
    marker=dict(color=CORES[0]),
))
fig.update_layout(title=dict(text="Saldo acumulado", automargin=True))
fig.update_xaxes(showgrid=False)
fig.show(config={"displayModeBar": False})

In [ ]:
# sem_dados
layout = pg.Layout(
    margin=dict(l=0, r=0, t=0, b=0),
    template="none",
    showlegend=False,
    xaxis=dict(
        showgrid=False,
        zeroline=False,
        visible=False),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        visible=False),
)
fig = pg.Figure(layout=layout)

# Adiciona anotação de texto
fig.add_annotation(
    x=0.5,
    y=0.5,
    xref="paper",
    yref="paper",
    text="Nenhum dado disponível",
    showarrow=False,
    font=dict(
        size=20
    )
)
fig.show(config={"displayModeBar": False})

In [ ]:
from cashd import plot
plot.saldo_acum("sem", 20)